In [1]:
import os
os.chdir('../')
%pwd

'e:\\MLOps\\DataScienceE2EProject1'

In [3]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/mohitsaini.nitk/DataScienceE2EProject1.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="mohitsaini.nitk"
os.environ["MLFLOW_TRACKING_PASSWORD"]="21ce4560d87a2cbe8ed23a2f893fbee85d073050"

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metric_file_name: Path
    all_params: dict
    target_column: str
    mlflow_uri: str

In [5]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories, save_json

In [6]:
class ConfigurationManager:

    def __init__(self, 
                config_filepath = CONFIG_FILE_PATH,
                params_filepath = PARAMS_FILE_PATH,
                schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            model_path=config.model_path,
            test_data_path=config.test_data_path,
            metric_file_name=config.metric_file_name,
            all_params=params,
            mlflow_uri="https://dagshub.com/mohitsaini.nitk/DataScienceE2EProject1.mlflow",
            target_column=schema.name
        )

        return model_evaluation_config



In [7]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import mlflow
from urllib.parse import urlparse
import mlflow.sklearn
import joblib

In [8]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config=config

    def eval_metrics(self,actual,pred):
        rmse=np.sqrt(mean_squared_error(actual,pred))
        mae=mean_absolute_error(actual,pred)
        r2=r2_score(actual,pred)

        return(rmse, mae, r2)

    def mlflow_logging(self):

        test_data=pd.read_csv(self.config.test_data_path)
        model=joblib.load(self.config.model_path)

        test_x=test_data.drop([self.config.target_column], axis=1)
        test_y=test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store=urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            predicting_qualities = model.predict(test_x)

            rmse, mae, r2 = self.eval_metrics(test_y, predicting_qualities)

            # Saving metrics local
            score={"rmse": rmse, "mae": mae, "r2": r2}
            save_json(Path(self.config.metric_file_name), data=score)

            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticNetModel")
            else:
                mlflow.sklearn.log_model(model, "model")



In [9]:
try:
    config=ConfigurationManager()
    model_evaluation_config=config.get_model_evaluation_config()
    model_evaluation=ModelEvaluation(config=model_evaluation_config)
    model_evaluation.mlflow_logging()
except Exception as e:
    raise e


[2025-07-05 03:02:31,552: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-05 03:02:31,554: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-05 03:02:31,558: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-07-05 03:02:31,559: INFO: common: Created directory: artifacts/model_evaluation]
[2025-07-05 03:02:32,833: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


2025/07/05 03:02:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'ElasticNetModel'.
2025/07/05 03:02:51 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticNetModel, version 1
Created version '1' of model 'ElasticNetModel'.


🏃 View run rumbling-sponge-440 at: https://dagshub.com/mohitsaini.nitk/DataScienceE2EProject1.mlflow/#/experiments/0/runs/62e393b591cc4acd96114ed27f39e71c
🧪 View experiment at: https://dagshub.com/mohitsaini.nitk/DataScienceE2EProject1.mlflow/#/experiments/0
